In [28]:
from mido import MidiFile
import sys
import json
import mido
import numpy as np
import random

def midifile_to_dict(mid):
    tracks = []
    for track in mid.tracks:
        tracks.append([vars(msg).copy() for msg in track])

    return {
        'ticks_per_beat': mid.ticks_per_beat,
        'tracks': tracks,
    }


def midi_to_txt(data_dir, file_name):
    song_name = file_name.replace('.mid','')
    json_path = data_dir + '/' + song_name
    txt_path = data_dir + '/' + song_name + '.txt'
    mid = mido.MidiFile(data_dir + '/' + file_name)

    #print(json.dumps(midifile_to_dict(mid), indent=2))
    with open(json_path, 'w') as outfile:
        json.dump(midifile_to_dict(mid), outfile)

    #The tempo is microseconds per beat, that is bpm = 60/(temp*10^(-6))
    #C4 is 60

    with open(json_path, 'r') as f:
         data = json.load(f)
    tpb = data['ticks_per_beat']
    note_events_number = len(data['tracks'][0]) #計算note event的數量，後面的index可能是0或是1，cubase輸出的midi是1
    note_events = data['tracks'][0] #取出note evnet的軌，後面的index可能是0或是1，cubase輸出的midi是1
    ticks_per_beat = data['ticks_per_beat']
    time_shift = 0
    ###Time shift start###
    for i in range(0,100):
        if note_events[i]['type'] == 'note_on':
            time_shift = note_events[i]['time']//ticks_per_beat*ticks_per_beat
            note_events[i]['time'] = note_events[i]['time'] - time_shift#刪除midi前面空白部分
            break
    ###Time shift end###

    ###將所有event的時間算成絕對時間 start###
    note_events_abs_time = []
    for i in range(0, note_events_number):
        note_events_abs_time.append(note_events[i]['time'])
    for i in range(0, note_events_number):
        if i != 0:
            note_events_abs_time[i] = note_events_abs_time[i-1] + note_events_abs_time[i]
    ###將所有event的時間算成絕對時間 end###


    ###note event to 2d-array start###
    #將note event存成note on 和note off的兩個2d-list，[[timestamp 1, pitch 1],....]
    note_on = []
    note_off = []
    note_check = []
    for i in range(0, note_events_number):
        if note_events[i]['type'] == 'note_on':
            note_on.append([note_events[i]['time'],note_events[i]['note']])
            note_check.append([note_events_abs_time[i],note_events[i]['note']])
        elif note_events[i]['type'] == 'note_off':
            note_off.append([note_events[i]['time'],note_events[i]['note']])
            note_check.append([note_events_abs_time[i],note_events[i]['note']])
    ###note event to 2d-array end### 

    ###檢查音有沒有重疊 start###
    note_check.sort(key=lambda x: x[0])
    note_check_pitch = np.array(note_check)[:,1]
    checktime = 0
    first = 0
    for idx in range(0,len(note_check_pitch)):
        checktime += 1
        if checktime == 1:
            first = note_check_pitch[idx]
        elif checktime == 2:
            if first != note_check_pitch[idx]:
                print('有重疊音')
                break
            checktime = 0
        if idx == len(note_check_pitch)-1:
            print('通過')
    ###檢查音有沒有重疊 end###

    midi_array = []
    condense_raio = 48
    condense_raio = (tpb//192)*condense_raio #normalize然後縮小ticks_per_beat，減少兩個note間的間隔，沒有縮小=1；設定為12時，最小note長度為1/64拍
    for note_idx in range(0, len(note_on)):
        for time_idx in range(0, note_on[note_idx][0]//condense_raio): #填入note on前的空白部分
            midi_array.append(90)
        midi_array.append(note_on[note_idx][1])
        for time_idx in range(0, note_off[note_idx][0]//condense_raio): #填入接續上一個音高，index為note pitch +128
            midi_array.append(89)
    

    with open(txt_path, 'w') as text_file:
        for idx in range(0, len(midi_array)):
            if idx != len(midi_array)-1:
                text_file.write(str(midi_array[idx]) + ' ')
            else:
                text_file.write(str(midi_array[idx]))
                
def concatenate_data(data_dir, txtname, input_string):
    output_data = ''
    txt_path = data_dir + '/' + txtname
    with open(txt_path, 'r') as text_file:
        output_data = input_string + ' ' + text_file.read()
    return output_data

def concatenate_txt(data_dir, files):
    pre_output_data = ''
    is_first_file = True
    for file in files:
        if file.find('.txt') != -1:
            if is_first_file == True:
                with open(data_dir + '/' + file, 'r') as text_file:
                    pre_output_data = text_file.read()
                is_first_file = False
            else:
                pre_output_data = concatenate_data(data_dir, file, pre_output_data)
    return pre_output_data

In [29]:
from os import listdir
from os.path import isfile, join
data_dir = 'midi_data'
all_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
for file in all_files:
    if file.find('.mid') != -1:
        midi_to_txt(data_dir, file)

#現在重疊音的處理方式為將該音的note off和重疊下一個音的note on的time stamp對調
#未來可改成將該音的note off縮到和下個音的note on一樣的time stamp

通過
有重疊音
通過
通過
有重疊音
通過
有重疊音
通過
通過
通過
通過
通過
通過
通過
有重疊音
通過
通過
通過
有重疊音
通過
有重疊音
通過
通過
通過
通過
通過
通過
通過
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
通過
通過
通過
有重疊音
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
通過
有重疊音
通過
通過
通過
通過
通過
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
通過
有重疊音
有重疊音
通過
有重疊音
有重疊音
通過
有重疊音
通過
通過
通過
通過
有重疊音
有重疊音
通過
通過
通過
有重疊音
通過
有重疊音
通過
有重疊音
通過
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
通過
通過
通過
通過
通過
有重疊音
有重疊音
通過
通過
有重疊音
有重疊音
有重疊音
通過
通過
通過
通過
通過
通過
有重疊音
通過
有重疊音
通過
通過
通過
通過
通過
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
通過
有重疊音
有重疊音
通過
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
通過
通過
有重疊音
通過
通過
通過
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
通過
通過
通過
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
通過
通過
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
有重疊音
通過
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
通過
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
有重疊音
通過
通過
通過
通過
通過
通過
通過
通過
通過
有重疊音
有重疊音
有重疊音
通

In [30]:
data_dir = 'midi_data'
all_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
random.shuffle(all_files)
all_txt = []
for file in all_files:
    if file.find('.txt') != -1:
        all_txt.append(file)
    
test_files = all_txt[0:len(all_txt)//12]
valid_files = all_txt[len(all_txt)//12 : 2*(len(all_txt)//12)]
train_files = all_txt[2*(len(all_txt)//12):len(all_txt)]

test_data = concatenate_txt(data_dir, test_files)
valid_data = concatenate_txt(data_dir, valid_files)
train_data = concatenate_txt(data_dir, train_files)
print(len(all_txt))
print(len(test_files))
print(len(valid_files))
print(len(train_files))
print(len(test_data))
print(len(valid_data))
print(len(train_data))

with open('midi_data/melodygen.test.txt', 'w') as text_file:
    text_file.write(test_data)
with open('midi_data/melodygen.valid.txt', 'w') as text_file:
    text_file.write(valid_data)
with open('midi_data/melodygen.train.txt', 'w') as text_file:
    text_file.write(train_data)

456
38
38
380
131507
179273
1393577


In [31]:
##去除多餘的空白，thre = 1/4拍

import sys

def despace(data_dir, txtname):
    with open(data_dir + "/" + txtname + ".txt", "r") as text_file:
        data = list(map(int, text_file.read().split(' ')))

    num_of_space = 0
    data_new = []
    for i in data:
        if i != 90:
            if num_of_space <= 4:
                for sp_n in range(0, num_of_space):
                    data_new.append(90)
                num_of_space = 0
            else:
                for sp_n in range(0, 4):
                    data_new.append(90)
                num_of_space = 0
            data_new.append(i)
        else:
            num_of_space += 1
    with open(data_dir + "/" + txtname + "_despace.txt", 'w') as text_file:
        for idx in range(0, len(data_new)):
            if idx != len(data_new)-1:
                text_file.write(str(data_new[idx]) + ' ')
            else:
                text_file.write(str(data_new[idx]))
data_dir = "data"
despace(data_dir, "melodygen.train")
despace(data_dir, "melodygen.valid")
despace(data_dir, "melodygen.test")
    

            
        

In [126]:
import random
a = [1,2,3,4,5,6,7,8,9,10]
print(len(a))
print(a[0:2])
print(a[2:4])
print(a[2*2:len(a)])

10
[1, 2]
[3, 4]
[5, 6, 7, 8, 9, 10]


In [165]:
#建立midi_idx
midi_onehot = []
for i in range(0, 91):
    midi_onehot.append(i)
with open('midi_data/melodygen.midi_idx.txt', 'w') as text_file:
        for idx in range(0, len(midi_onehot)):
            if idx != len(midi_onehot)-1:
                text_file.write(str(midi_onehot[idx]) + ' ')
            else:
                text_file.write(str(midi_onehot[idx]))

In [2]:
a = [1,2,3]
for i in range(0, len(a)):
    print(i)

0
1
2


In [3]:
data_dir = 'data/'
file = 'c0130.mid'
midi_to_txt(data_dir, file)

有重疊音


IndexError: list index out of range

In [26]:
import numpy as np
import math
np.log(math.exp( 1 ))

1.0

In [27]:
decay_rate = 0.9
4.60517//np.log(decay_rate)

-44.0